# Flow Metrics

This workbook generates reports on flow metric from a given GitHub repo using the Github API.

Author: [Brian McIlwain](mailto:brian@poq.gg)

## Setup

This gets data from the [Github GraphQL API](https://docs.github.com/en/graphql/overview/about-the-graphql-api) to create reports for flow metrics

### Imports, utils, & globals

In [1]:
from dateutil.relativedelta import relativedelta, SU
from datetime import date
from functools import reduce
import pandas as pd
from gql.transport.aiohttp import AIOHTTPTransport
from gql import gql, Client
from os import environ
from dotenv import load_dotenv

load_dotenv()
API_SECRET_TOKEN = environ.get('API_SECRET_TOKEN')

if not API_SECRET_TOKEN:
    raise Exception(
        'API_SECRET_TOKEN is not defined as an environmental variable')

# Config for repo
API_URL = "https://api.github.com/graphql"
REPO_OWNER = "weiks"
REPO_NAME = "esports-backend"
MAX_WEEK_ISSUES = 100  # API limit, shouldn't be hit
MAX_PAGINATION_LIMIT = 100  # Set by API, I wish it was larger
MAX_LABEL_LIMIT = 20  # Max labels per PR

print(f"Repo: {REPO_OWNER}/{REPO_NAME}\n")

# Prep to run GQL calls
headers = {"Authorization": f"Bearer {API_SECRET_TOKEN}"}
transport = AIOHTTPTransport(url=API_URL, headers=headers)


async def runQuery(query, variable_values=None):
    # Create a GraphQL client using the defined transport
    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:
        # Execute the query on the transport
        result = await session.execute(query, variable_values)
        return result


def get_previous_sunday(working_date=date.today()):
    last_sunday = working_date + relativedelta(weekday=SU(-1))
    return last_sunday.strftime("%Y-%m-%d")


def unwrap_name(d):
    return d['name']


def unwrap_login(d):
    return d['login']


def clean_issue_data(issue):
    issue['author'] = issue['author']['login']
    issue['labels'] = set(map(unwrap_name, issue['labels']['nodes']))
    issue['assignees'] = set(map(unwrap_login, issue['assignees']['nodes']))

    return issue


def clean_PR_data(pr):
    pr['author'] = pr['author']['login']
    pr['assignees'] = set(map(unwrap_login, pr['assignees']['nodes']))
    pr['assignees'].add(pr['author'])  # Author is an assignee by default
    pr['issues'] = list(map(clean_issue_data, pr['issues']['nodes']))
    pr['labels'] = set(map(unwrap_name, pr['labels']['nodes']))
    pr['labels'].discard('trigger-ci')  # Ignore trigger-ci

    # Apply parent issue labels to the PR
    if pr['issues']:
        issues_labels = list(map(lambda issue: issue['labels'], pr['issues']))
        flat_issues_labels = reduce(lambda a, b: a.union(b), issues_labels)
        pr['labels'] = pr['labels'].union(flat_issues_labels)

    return pr


Repo: weiks/esports-backend



## Get PRs Data

In [2]:
async def get_PR_data():
    cursor = ""
    prs = []
    print('Fetching PRs, this can take a while...')

    while cursor != None:
        overviewQuery = gql(
            f"""
            query getRepoData {{
              repository(owner: "{REPO_OWNER}", name: "{REPO_NAME}") {{
                pullRequests(
                  first: {MAX_PAGINATION_LIMIT},
                  orderBy: {{ field: UPDATED_AT, direction: DESC }}
                  { f'after: "{cursor}"' if cursor else '' }
                ) {{
                  pageInfo {{
                    hasNextPage
                    endCursor
                  }}
                  nodes {{
                    title
                    author {{
                      login
                    }}
                    assignees(first: 10) {{
                      nodes {{
                        login
                      }}
                    }}
                    number
                    url
                    state
                    closedAt
                    createdAt
                    updatedAt
                    labels(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        name
                      }}
                    }}
                    issues: closingIssuesReferences(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        assignees(first: 10) {{
                          nodes {{
                            login
                          }}
                        }}
                        labels(first: {MAX_LABEL_LIMIT}) {{
                          nodes {{
                            name
                          }}
                        }}
                        number
                        title
                        url
                        state
                        updatedAt
                        author {{
                          login
                        }}
                      }}
                    }}
                  }}
                }}
              }}
            }}
            """)
        result = await runQuery(overviewQuery)
        prs += result['repository']['pullRequests']['nodes']
        cursor = result['repository']['pullRequests']['pageInfo']['endCursor']
    prs = list(map(clean_PR_data, prs))
    prsDF = pd.DataFrame(prs)
    prsDF.to_csv('prs_data.csv', index=False)
    return prsDF

await get_PR_data()


Fetching PRs, this can take a while...


,title,author,assignees,number,url,state,closedAt,createdAt,updatedAt,labels,issues
0,Paypal payment method,mikiasyonas,{mikiasyonas},1732,https://github.com/weiks/esports-backend/pull/...,OPEN,None,2022-06-30T20:27:06Z,2022-07-11T21:13:52Z,"{enhancement, good for newcomers, interview, b...","[{'assignees': {'mikiasyonas'}, 'labels': {'in..."
1,Fix normal event announcement not being saved ...,mikiasyonas,{mikiasyonas},1761,https://github.com/weiks/esports-backend/pull/...,OPEN,None,2022-07-08T10:30:13Z,2022-07-11T19:26:28Z,"{critical, bug}","[{'assignees': {'mikiasyonas'}, 'labels': {'cr..."
2,[ Fix ] Verify join modal i18n,brianmazzocchi,{brianmazzocchi},1774,https://github.com/weiks/esports-backend/pull/...,MERGED,2022-07-11T18:14:37Z,2022-07-11T16:15:30Z,2022-07-11T18:14:37Z,{},[]
3,Update changeStatus to send teams in bulk,Mathspy,{Mathspy},1773,https://github.com/weiks/esports-backend/pull/...,MERGED,2022-07-11T16:31:11Z,2022-07-11T15:53:41Z,2022-07-11T16:31:12Z,{},[]
4,Doc: Tech debt plan,bmcilw1,{bmcilw1},1703,https://github.com/weiks/esports-backend/pull/...,OPEN,None,2022-06-23T16:14:20Z,2022-07-11T16:26:36Z,{docs},[]
...,...,...,...,...,...,...,...,...,...,...,...
1446,Add auth middleware to discord/token endpoint,Mathspy,{Mathspy},6,https://github.com/weiks/esports-backend/pull/6,MERGED,2020-03-18T01:17:28Z,2020-03-18T01:16:02Z,2020-03-18T01:17:34Z,{},[]
1447,Add users to db on successful OAuth,Mathspy,{Mathspy},5,https://github.com/weiks/esports-backend/pull/5,MERGED,2020-03-17T22:39:52Z,2020-03-17T22:33:44Z,2020-03-17T22:39:55Z,{},[]
1448,Swapped REDIRECT_URI to be env var based,Mathspy,{Mathspy},4,https://github.com/weiks/esports-backend/pull/4,MERGED,2020-03-14T21:08:57Z,2020-03-14T21:00:49Z,2020-03-14T21:09:00Z,{},[]
1449,Added new slot fields required by reworked qBonus,Mathspy,{Mathspy},2,https://github.com/weiks/esports-backend/pull/2,MERGED,2020-02-15T15:44:36Z,2020-02-15T15:27:31Z,2020-02-18T00:27:06Z,{},[]


## Get Issue Data

In [3]:
async def get_issue_data():
    cursor = ""
    issues = []
    print('Fetching issues, this can take a while...')

    while cursor != None:
        overviewQuery = gql(
            f"""
            query getRepoData {{
              repository(owner: "{REPO_OWNER}", name: "{REPO_NAME}") {{
                issues(
                  first: {MAX_WEEK_ISSUES}, 
                  orderBy: {{ field: UPDATED_AT, direction: DESC }}
                  { f'after: "{cursor}"' if cursor else '' }
                ) {{
                  pageInfo {{
                    hasNextPage
                    endCursor
                  }}
                  nodes {{
                    title
                    author {{
                      login
                    }}
                    assignees(first: 10) {{
                      nodes {{
                        login
                      }}
                    }}
                    number
                    url
                    state
                    closedAt
                    createdAt
                    updatedAt
                    labels(first: {MAX_LABEL_LIMIT}) {{
                      nodes {{
                        name
                      }}
                    }}
                  }}
                }}
              }}
            }}
            """)
        result = await runQuery(overviewQuery)
        issues += result['repository']['issues']['nodes']
        cursor = result['repository']['issues']['pageInfo']['endCursor']
        # cursor = None # Comment out this line to fetch all
    issues = list(map(clean_issue_data, issues))
    issuesDF = pd.DataFrame(issues)
    issuesDF.to_csv('issues_data.csv', index=False)
    return issuesDF


await get_issue_data()


Fetching issues, this can take a while...


,title,author,assignees,number,url,state,closedAt,createdAt,updatedAt,labels
0,[Enhancement] Add support for buying Quarters ...,bmcilw1,{mikiasyonas},1417,https://github.com/weiks/esports-backend/issue...,OPEN,None,2022-05-09T18:08:43Z,2022-07-11T21:09:39Z,"{interview, good for newcomers, enhancement, b..."
1,[ Bug ] Getting Error 500 / 503 when `guilds` ...,brianmazzocchi,{},1776,https://github.com/weiks/esports-backend/issue...,OPEN,None,2022-07-11T21:05:57Z,2022-07-11T21:05:57Z,{critical}
2,[Model] Add data model for Challonge account c...,alexangc,{},1775,https://github.com/weiks/esports-backend/issue...,OPEN,None,2022-07-11T17:24:52Z,2022-07-11T20:36:19Z,{}
3,[Milestone] Swap War Legends (WLS) to Yunite f...,bmcilw1,"{alexangc, Mathspy}",1061,https://github.com/weiks/esports-backend/issue...,OPEN,None,2022-03-22T18:04:30Z,2022-07-11T20:32:52Z,{enhancement}
4,Provide a way for hosts to update their Discor...,alexangc,{gonzalovelasco},1762,https://github.com/weiks/esports-backend/issue...,CLOSED,2022-07-11T20:30:57Z,2022-07-08T14:06:39Z,2022-07-11T20:30:57Z,{enhancement}
...,...,...,...,...,...,...,...,...,...,...
320,[Airbrake] [Production] 10 ABORTED: The refere...,weiks,{Mathspy},527,https://github.com/weiks/esports-backend/issue...,CLOSED,2021-03-31T14:43:40Z,2021-03-11T08:16:28Z,2021-03-31T14:43:40Z,{}
321,[Airbrake] [Production] Cannot read property '...,weiks,{},392,https://github.com/weiks/esports-backend/issue...,CLOSED,2021-01-23T20:47:12Z,2021-01-23T17:12:17Z,2021-01-23T20:47:12Z,{}
322,"[Airbrake] [Production] Value for argument ""do...",weiks,{},391,https://github.com/weiks/esports-backend/issue...,CLOSED,2021-01-23T20:38:37Z,2021-01-23T16:48:08Z,2021-01-23T20:38:37Z,{}
323,[Airbrake] [Production] Channel closed by serv...,weiks,{},334,https://github.com/weiks/esports-backend/issue...,CLOSED,2020-12-29T19:42:18Z,2020-12-29T09:12:59Z,2020-12-29T19:42:18Z,{}
